# Estudo de oportunidades para PBZPA

In [1]:
# helper function
from os import listdir
def dir_files(name: str, folder: str = None):
    if folder:
        files = [folder + e for e in listdir(folder) if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    else:
        files = [e for e in listdir() if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    print(f"{name}: {', '.join(files)}")
    return files

In [3]:
from pandas import read_excel
from pandas import read_csv

public_private = dir_files('Aeródromos públicos e privados', r'E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/')
aerodromos_publicos =  read_excel(public_private[1], sheet_name=0, header=[1,2])
aerodromos_privados = read_excel(public_private[0], sheet_name=0, header=[0,1])

print('\n')

aisweb = dir_files('AISWeb')
aisweb =  read_csv(aisweb[0], sep=',', encoding='latin-1')

aisweb.insert(loc=4, column="Categoria",value='')
icao_aeropublicos = aerodromos_publicos.iloc[:,0].values
icao_aeroprivados = aerodromos_privados.iloc[:,0].values
for i, icao in enumerate(aisweb.ICAO):
    if icao in icao_aeropublicos:
        aisweb.at[i, 'Categoria'] = 'Público'
    elif icao in icao_aeroprivados:
        aisweb.at[i, 'Categoria'] = 'Privado'

Aeródromos públicos e privados: E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Privados_28MAR2019.xls, E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Públicos_28MAR2019.xls


AISWeb: output_aisweb_4428c205cd.csv


In [4]:
aisweb.count()

timestamp    2630
ICAO         2630
CIAD         2569
Aeródromo    2568
Categoria    2630
Cidade       2568
UF           2568
Status       2630
NOTAM        2630
Debug        2630
dtype: int64

### Há valores nulos?

In [5]:
aisweb.isnull().sum()

timestamp     0
ICAO          0
CIAD         61
Aeródromo    62
Categoria     0
Cidade       62
UF           62
Status        0
NOTAM         0
Debug         0
dtype: int64

### Aeródromos não encontrados (fechado, localidade cancelada, Localidade reservada aguardando portaria da ANAC)

#### Aeródromos que constam na lista da ANAC, mas não foram encontrados no AISWeb

In [ ]:
aero_naoencontrado = aisweb[aisweb.Status == "O aeródromo não foi encontrado"]
print(f"Total de aeródromos com Status: {aero_naoencontrado.ICAO.count()}")

#### Localidade cancelada

In [ ]:
aero_localizacao_cancelada = aisweb[aisweb.Status == "Localidade cancelada"]
print(f"Total de aeródromos ({aero_localizacao_cancelada.Status[:1].item()}): {aero_localizacao_cancelada.ICAO.count()}")

#### Aeródromos com localidade reservada aguardando portaria da ANAC

In [ ]:
aero_localizacao_reservada = aisweb[aisweb.Status == "Localidade reservada. Aguardando portaria da ANAC."]
print(f"Total de aeródromos ({aero_localizacao_reservada.Status[:1].item()}): {aero_localizacao_reservada.ICAO.count()}")

#### Aeródromos fechados até determinada data

In [ ]:
filtro = "Aeródromo FECHADO até"
aero_fechado = aisweb[aisweb.Status.str.contains(filtro) == True]
print(f"Total de aeródromos fechados: {aero_fechado.ICAO.count()}")

#### Aeródromos com dados faltando
Estão iniciando atividade? ICAO consta no AISWeb, não foi exibida nenhuma mensagem de Status.

In [ ]:
aero_null = aisweb[aisweb.Cidade.isnull()]
print(f"Total de aeródromos com valores nulos: {aero_null.ICAO.count()}")

In [ ]:
aero_null.isnull().sum()

In [ ]:
aero_null.head()

## NOTAM

#### Aeródromos contendo a string "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"

In [6]:
filtro = "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"
ad_clsd_pbzpa = aisweb[aisweb.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO\": {ad_clsd_pbzpa.ICAO.count()}")

Total de aeródromos com "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO": 193


In [7]:
ad_clsd_pbzpa.head()

,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,NOTAM,Debug
23,2019-04-18T04:10:27.345320-03:00,SDIO,SP0109,Aeroclube de Itápolis,Privado,ITÁPOLIS,SP,OK,['Revogado INFOTEMP 1090/2018 \n AD CLSD DEVID...,{'response_status_code': 200}
54,2019-04-18T04:12:50.386320-03:00,SIVD,MS0134,Agropecuária Santa Rita,Privado,RIO VERDE DE MATO GROSSO,MS,Aeródromo FECHADO até 30/06/19: INFOTEMP 1486/...,['INFOTEMP 1486/2018 \n AD CLSD DEVIDO NAO APR...,{'response_status_code': 200}
61,2019-04-18T04:09:14.462284-03:00,SJYE,RR0089,Água Fria,Privado,UIRAMUTÃ,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1284/...,['INFOTEMP 1284/2018 \n AD CLSD DEVIDO NAO AP...,{'response_status_code': 200}
63,2019-04-18T04:09:23.784286-03:00,SJQL,MG0130,Águas Claras Aviação Agrícola,Privado,SANTA JULIANA,MG,OK,['INFOTEMP 0199/2019 \n CIRCUITO DE TFC PARA A...,{'response_status_code': 200}
65,2019-04-18T04:09:31.812286-03:00,SJYF,RR0090,Ajarani,Privado,CARACARAÍ,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1213/...,['INFOTEMP 1213/2018 \n AD CLSD DEVIDO NAO AP...,{'response_status_code': 200}


#### Com fechamento não revogado ou está agendado para fechamento

In [ ]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
oportunidades = ad_clsd_pbzpa[~ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True) & ad_clsd_pbzpa.Status.str.contains('Aeródromo FECHADO')]
print(f"Total de prospects para novos negócios: {oportunidades.ICAO.count()}")

#### Revogado o fechamento

In [ ]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
ad_clsd_pbzpa_rev = ad_clsd_pbzpa[ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True)]
print(f"Total de aeródromos com \"AD CLSD\" revogado: {ad_clsd_pbzpa_rev.ICAO.count()}")

#### Com fechamento revogado, mas somente em missões de caráter emergencial
ou seja, candidatos para INFRACEA | Social. Da retorno de mídia. Será que há empresas com interesse na facilitação de acesso à região? Será há benefício fiscal para empresas que financiarem o produto PBZPA da INFRACEA nestas regiões?

In [ ]:
filtro = "CARATER EMERGENCIAL"
ad_clsd_pbzpa_rev_emg = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD\" revogado em caráter emergencial: {ad_clsd_pbzpa_rev_emg.ICAO.count()}")

#### Revogação do fechamento ratificada
situação inferida: gestão fraca? Estão desesperados esperando a INFRACEA | Aeroportos?

In [ ]:
ad_clsd_pbzpa_rev_ratif = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.ICAO.isin(aero_fechado.ICAO)]
print(f"Total de aeródromos com funcionamento turbulento: {ad_clsd_pbzpa_rev_ratif.ICAO.count()}")

##### Exportar para excel

In [ ]:
#ad_clsd_pbzpa.to_excel('notam_ad_clsd_pbzpa.xlsx', index=False, na_rep='')